# Task 1: In-Silico Perturbation Workflow

- Rank-based swaps (`knock_rank`)
- Expression-based multiplicative change with optional propagation (`knock_expr`)

In [1]:
# Ensure repo root is on sys.path for `utils` imports
from pathlib import Path
import sys
repo_root = Path.cwd().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))
print('Added to sys.path:', repo_root)


Added to sys.path: /cs/student/projects1/aibh/2024/rmaheswa/Helical_Task/als-perturb-geneformer


In [2]:
import os
os.environ["PIP_CACHE_DIR"] = "/cs/student/projects1/aibh/2024/rmaheswa/cache"
os.environ["HF_HOME"] = "/cs/student/projects1/aibh/2024/rmaheswa/cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/cs/student/projects1/aibh/2024/rmaheswa/cache/transformers"

In [3]:
# Intro & setup
from pathlib import Path
import logging
import numpy as np
import scanpy as sc

from utils.io import ensure_dirs, read_raw, write_adata
from utils.perturb import knock_rank, knock_expr

logging.basicConfig(level=logging.INFO)
np.random.seed(42)

ensure_dirs()

RAW_DIR = Path('/cs/student/projects1/aibh/2024/rmaheswa/Helical_Task/als-perturb-geneformer/als-perturb-geneformer/data/raw')
AD_DIR = Path('/cs/student/projects1/aibh/2024/rmaheswa/Helical_Task/als-perturb-geneformer/als-perturb-geneformer/data/adata')
FIG_DIR = Path('/cs/student/projects1/aibh/2024/rmaheswa/Helical_Task/als-perturb-geneformer/als-perturb-geneformer/data/figs')

AD_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print('If baseline.h5ad is not present, run: make prep')


If baseline.h5ad is not present, run: make prep


In [4]:
# Load baseline
from pathlib import Path
import anndata as ad

baseline_path = AD_DIR / 'baseline.h5ad'
if not baseline_path.exists():
    raise FileNotFoundError("Missing data/adata/baseline.h5ad. Please run: make prep")

adata = ad.read_h5ad(baseline_path)
print(f"Loaded AnnData: {adata.n_obs} cells, {adata.n_vars} genes")

# Test with a smaller subset to avoid kernel crashes
print("Subsetting to first 1000 cells for testing...")
adata = adata[:1000].copy()
print(f"Subset AnnData: {adata.n_obs} cells, {adata.n_vars} genes")
adata


Loaded AnnData: 112014 cells, 22026 genes
Subsetting to first 1000 cells for testing...
Subset AnnData: 1000 cells, 22026 genes


AnnData object with n_obs × n_vars = 1000 × 22026
    obs: 'Sample_ID', 'Donor', 'Region', 'Sex', 'Condition', 'Group', 'C9_pos', 'CellClass', 'CellType', 'SubType', 'full_label', 'DGE_Group', 'Bakken_M1', 'data_merge_id', 'data_sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'Cellstates_LVL1', 'Cellstates_LVL2', 'Cellstates_LVL3', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_genes', 'split'
    var: 'Biotype', 'Chromosome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'ENSID', 'mt', 'n_cells', 'biotype', 'gene_symbol'
    uns: 'log1p'
    layers: 'counts'

In [5]:
# Demonstrate rank-mode perturbation (single-gene up/down)
import gc

gene_demo = [adata.var['gene_symbol'][0]] if adata.n_vars > 0 else ['G0']
print(f"Testing with gene: {gene_demo[0]}")

print("Creating rank-up perturbation...")
adata_rank_up = knock_rank(adata, genes=gene_demo, direction='up', delta_percentile=0.15)

print("Creating rank-down perturbation...")
adata_rank_down = knock_rank(adata, genes=gene_demo, direction='down', delta_percentile=0.15)

# Validate bounds
ranks = adata_rank_up.obsm['rank_matrix']
assert ranks is None or (ranks.min() >= 0 and ranks.max() < adata.n_vars)

print('Rank-mode demo complete.')
gc.collect()  # Force garbage collection


Testing with gene: TSPAN6
Creating rank-up perturbation...


/tmp/ipykernel_2334960/3992818746.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gene_demo = [adata.var['gene_symbol'][0]] if adata.n_vars > 0 else ['G0']


Creating rank-down perturbation...
Rank-mode demo complete.


1367

In [6]:
# Demonstrate expression-mode perturbation (single & multi-gene)
import numpy as np

print("Creating single-gene expression perturbation...")
# Single gene up-regulation without propagation
adata_expr_up = knock_expr(adata, genes=gene_demo, direction='up', log2fc=1.0, propagate=False)

# Library size preserved
lib_before = np.array(adata.layers['counts'].sum(axis=1)).flatten()
lib_after = np.array(adata_expr_up.X.sum(axis=1)).flatten()
np.testing.assert_allclose(lib_before, lib_after, rtol=1e-5, atol=1e-5)
print("✓ Library size preserved")

print("Creating multi-gene expression perturbation...")
# Multi-gene with propagation
multi_genes = list(adata.var['gene_symbol'][:5])
print(f"Multi-gene test with: {multi_genes}")
adata_expr_multi = knock_expr(adata, genes=multi_genes, direction='down', log2fc=0.5, propagate=True, k_neighbors=20)

print('Expression-mode demo complete.')
gc.collect()  # Force garbage collection


Creating single-gene expression perturbation...
✓ Library size preserved
Creating multi-gene expression perturbation...
Multi-gene test with: ['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'FGR']
Expression-mode demo complete.


0

In [7]:
# Save perturbed outputs
from utils.io import write_adata

write_adata(adata_rank_up, AD_DIR / 'pert_rank_up.h5ad')
write_adata(adata_rank_down, AD_DIR / 'pert_rank_down.h5ad')
write_adata(adata_expr_up, AD_DIR / 'pert_expr_single_up.h5ad')
write_adata(adata_expr_multi, AD_DIR / 'pert_expr_multi_down.h5ad')
print('Saved perturbed AnnData files.')


Saved perturbed AnnData files.


In [8]:
# Export simple workflow schematic figure
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(6,3))
plt.text(0.05, 0.6, 'Raw counts', fontsize=12)
plt.arrow(0.2, 0.6, 0.2, 0.0, head_width=0.02, length_includes_head=True)
plt.text(0.42, 0.6, 'QC + norm', fontsize=12)
plt.arrow(0.57, 0.6, 0.2, 0.0, head_width=0.02, length_includes_head=True)
plt.text(0.79, 0.6, 'Perturb\n(rank/expr)', fontsize=12)
plt.axis('off')
plt.tight_layout()
plt.savefig(FIG_DIR / 'task1_workflow.png', dpi=200)
plt.close()

print('Saved figure to /cs/student/projects1/aibh/2024/rmaheswa/Helical_Task/als-perturb-geneformer/als-perturb-geneformer/data/figs/task1_workflow.png')


Saved figure to /cs/student/projects1/aibh/2024/rmaheswa/Helical_Task/als-perturb-geneformer/als-perturb-geneformer/data/figs/task1_workflow.png
